In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd

from blooms_ml.utils import labeling

In [ ]:
df_stat = pd.read_csv(f"{Path.home()}/data_ROHO/cnps_mean_std.csv")
df = pd.read_parquet(f"{Path.home()}/data_ROHO/roho800_weekly_average.parquet")

In [ ]:
df_stat

In [ ]:
df = df.groupby(['station', 's_rho']).apply(labeling, include_groups=False)
df = df.reset_index().drop(columns='level_2')

In [ ]:
df.rename(columns={'label': 'y'}, inplace=True)
df['label'] = np.where(df['y'] > 1, 1, 0)

In [ ]:
df

In [ ]:
df_station = df[df['station'] == 0][df['s_rho'] == -0.02]
df_station.set_index('ocean_time', inplace=True)
df_station

In [ ]:
df_station['label'].plot(figsize=(14, 7))

In [ ]:
df_station['P1_c'].plot(figsize=(14, 7))

In [ ]:
df_station['N5_s'].plot(figsize=(14, 7))